In [1]:
import gymnasium as gym
gym.make('CartPole-v1')

<TimeLimit<OrderEnforcing<PassiveEnvChecker<CartPoleEnv<CartPole-v1>>>>>

In [2]:
import torch 

In [ ]:
 a = [True,  True, False, True]

In [29]:
sprobs = torch.softmax(torch.tensor([5.0,1.,1.]), -1)
sprobs

tensor([0.9647, 0.0177, 0.0177])

In [31]:
lsprobs = torch.log_softmax(torch.tensor([5.0,1.,1.]), -1)
lsprobs

tensor([-0.0360, -4.0360, -4.0360])

In [30]:
# c = torch.distributions.Categorical(torch.tensor([5.0,1.,1.]))
c = torch.distributions.Categorical(sprobs)
c.probs

tensor([0.9647, 0.0177, 0.0177])

In [8]:
a = torch.randn((1,5))
a.detach()

tensor([[-0.3827, -0.0586,  0.0845,  0.4314,  2.2272]])

In [4]:
9.0 ** 2

81.0

In [22]:
a = set([1, 3, 5])
b = set([1, 3 , 5, 6])

print(a)
print(b)
print(a == b)
print(b | a)
print(a - b)

{1, 3, 5}
{1, 3, 5, 6}
False
{1, 3, 5, 6}
set()


## Quicker Gameplay

Going to see if we can improve a gamestep by making all the actions happen at once


In [ ]:
@torch.no_grad()
def play_recurrent_game(env, wolf_policy, villager_agent, num_times=10, hidden_state_size=None, voting_type=None):
    
    wins = 0
    # loop = tqdm(range(num_times))
    for _ in range(num_times):
        ## Play the game 
        next_observations, rewards, terminations, truncations, infos = env.reset()
        # init recurrent stuff for actor and critic to 0 as well
        magent_obs = {agent: {'obs': [], 
                              # obs size, and 1,1,64 as we pass batch first
                              'hcxs': [(torch.zeros((1,1,hidden_state_size), dtype=torch.float32), torch.zeros((1,1,hidden_state_size), dtype=torch.float32))],
                    } for agent in env.agents if not env.agent_roles[agent]}
    
        wolf_action = None
        while env.agents:
            observations = copy.deepcopy(next_observations)
            actions = {}

            villagers = set(env.agents) & set(env.world_state["villagers"])
            wolves = set(env.agents) & set(env.world_state["werewolves"])

            # villagers actions
            for villager in villagers:
                #torch.tensor(env.convert_obs(observations['player_0']['observation']), dtype=torch.float)
                torch_obs = torch.tensor(env.convert_obs(observations[villager]['observation']), dtype=torch.float)
                obs = torch.unsqueeze(torch_obs, 0)

                # TODO: Testing this, we may need a better way to pass in villagers
                recurrent_cell = magent_obs[villager]["hcxs"][-1]
                
                # ensure that the obs is of size (batch,seq,inputs)
                policies, _, recurrent_cell = villager_agent(obs, recurrent_cell)
                _, game_action = villager_agent.get_action_from_policies(policies, voting_type=voting_type)

                if voting_type == "plurality":
                    actions[villager] = game_action.item()
                elif voting_type == "approval":
                    actions[villager] = game_action.tolist()

                #store the next recurrent cells
                magent_obs[villager]["hcxs"].append(recurrent_cell)

            # wolf steps
            phase = env.world_state['phase']
            for wolf in wolves:
                wolf_action = wolf_policy(env, wolf, action=wolf_action)
                actions[wolf] = wolf_action
        
            next_observations, _, _, _, _ = env.step(actions)
            
            # clear the wolf action if needed
            if env.world_state['phase'] == Phase.NIGHT:
                wolf_action = None
            
            if env.world_state['phase'] == Phase.ACCUSATION and phase == Phase.NIGHT:
                wolf_action = None

        ## Fill bigger buffer, keeping in mind sequence
        winner = env.world_state['winners']
        if winner == Roles.VILLAGER:
            wins += 1

        # loop.set_description(f"Villagers won {wins} out of a total of {num_times} games")
    
    return wins